# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), May 15, 2021**

In [1]:
using DataFrames # load package

In [2]:
using CategoricalArrays # CategoricalArrays.jl is independent from DataFrames.jl but it is often used in combination

## Working with CategoricalArrays

### Constructor

In [3]:
x = categorical(["A", "B", "B", "C"]) # unordered

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [4]:
y = categorical(["A", "B", "B", "C"], ordered=true) # ordered, by default order is sorting order

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [5]:
z = categorical(["A","B","B","C", missing]) # unordered with missings

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"
 "B"
 "B"
 "C"
 missing

In [6]:
c = cut(1:10, 5) # ordered, into equal counts, possible to rename labels and give custom breaks

10-element CategoricalArray{String,1,UInt32}:
 "Q1: [1.0, 2.8)"
 "Q1: [1.0, 2.8)"
 "Q2: [2.8, 4.6)"
 "Q2: [2.8, 4.6)"
 "Q3: [4.6, 6.4)"
 "Q3: [4.6, 6.4)"
 "Q4: [6.4, 8.2)"
 "Q4: [6.4, 8.2)"
 "Q5: [8.2, 10.0]"
 "Q5: [8.2, 10.0]"

(we will cover grouping later, but let us here use it to analyze the results, we use Pipe.jl for chaining)

In [7]:
using Pipe

In [8]:
@pipe DataFrame(x=cut(randn(100000), 10)) |>
      groupby(_, :x) |>
      combine(_, nrow) # just to make sure cut works right

,x,nrow
,Cat…,Int64
1,"Q1: [-4.578720462676443, -1.2873996729966137)",10000
2,"Q2: [-1.2873996729966137, -0.8471097681546838)",10000
3,"Q3: [-0.8471097681546838, -0.5322836874713975)",10000
4,"Q4: [-0.5322836874713975, -0.25977937584252303)",10000
5,"Q5: [-0.25977937584252303, -0.0030385208169451533)",10000
6,"Q6: [-0.0030385208169451533, 0.2509326578365592)",10000
7,"Q7: [0.2509326578365592, 0.5228764428793446)",10000
8,"Q8: [0.5228764428793446, 0.8366276730375477)",10000
9,"Q9: [0.8366276730375477, 1.280108145583436)",10000


In [9]:
v = categorical([1,2,2,3,3]) # contains integers not strings

5-element CategoricalArray{Int64,1,UInt32}:
 1
 2
 2
 3
 3

In [10]:
Vector{Union{String, Missing}}(z) # sometimes you need to convert back to a standard vector

5-element Vector{Union{Missing, String}}:
 "A"
 "B"
 "B"
 "C"
 missing

### Managing levels

In [11]:
arr = [x,y,z,c,v]

5-element Vector{CategoricalVector{T, UInt32, V, C, U} where {T, V, C, U}}:
 CategoricalValue{String, UInt32}["A", "B", "B", "C"]
 CategoricalValue{String, UInt32}["A", "B", "B", "C"]
 Union{Missing, CategoricalValue{String, UInt32}}["A", "B", "B", "C", missing]
 CategoricalValue{String, UInt32}["Q1: [1.0, 2.8)", "Q1: [1.0, 2.8)", "Q2: [2.8, 4.6)", "Q2: [2.8, 4.6)", "Q3: [4.6, 6.4)", "Q3: [4.6, 6.4)", "Q4: [6.4, 8.2)", "Q4: [6.4, 8.2)", "Q5: [8.2, 10.0]", "Q5: [8.2, 10.0]"]
 CategoricalValue{Int64, UInt32}[1, 2, 2, 3, 3]

In [12]:
isordered.(arr) # chcek if categorical array is orderd

5-element BitVector:
 0
 1
 0
 1
 0

In [13]:
ordered!(x, true), isordered(x) # make x ordered

(CategoricalValue{String, UInt32}["A", "B", "B", "C"], true)

In [14]:
ordered!(x, false), isordered(x) # and unordered again

(CategoricalValue{String, UInt32}["A", "B", "B", "C"], false)

In [15]:
levels.(arr) # list levels

5-element Vector{Vector{T} where T}:
 ["A", "B", "C"]
 ["A", "B", "C"]
 ["A", "B", "C"]
 ["Q1: [1.0, 2.8)", "Q2: [2.8, 4.6)", "Q3: [4.6, 6.4)", "Q4: [6.4, 8.2)", "Q5: [8.2, 10.0]"]
 [1, 2, 3]

In [16]:
unique.(arr) # missing will be included

5-element Vector{Vector{T} where T}:
 ["A", "B", "C"]
 ["A", "B", "C"]
 Union{Missing, String}["A", "B", "C", missing]
 ["Q1: [1.0, 2.8)", "Q2: [2.8, 4.6)", "Q3: [4.6, 6.4)", "Q4: [6.4, 8.2)", "Q5: [8.2, 10.0]"]
 [1, 2, 3]

In [17]:
y[1] < y[2] # can compare as y is ordered

true

In [18]:
v[1] < v[2] # not comparable, v is unordered although it contains integers

LoadError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [19]:
y[2] < "A" # comparison against type underlying categorical value is not allowed

LoadError: ArgumentError: cannot compare a `CategoricalValue` to value `v` of type `CategoricalValue{String, UInt32}`: wrap `v` using `CategoricalValue(v, catvalue)` or `CategoricalValue(v, catarray)` first

In [20]:
y[2] < CategoricalValue("A", y) # you need to explicitly convert a value to a level

false

In [21]:
y[2] < CategoricalValue("Z", y) # but it is treated as a level, and thus only valid levels are allowed

LoadError: ArgumentError: level Z not found in source pool

In [22]:
levels!(y, ["C", "B", "A"]) # you can reorder levels, mostly useful for ordered CategoricalArrays

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [23]:
y[1] < y[2] # observe that the order is changed

false

In [24]:
levels!(z, ["A", "B"]) # you have to specify all levels that are present

LoadError: ArgumentError: cannot remove level "C" as it is used at position 4 and allowmissing=false.

In [25]:
levels!(z, ["A", "B"], allowmissing=true) # unless the underlying array allows for missings and force removal of levels

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"
 "B"
 "B"
 missing
 missing

In [26]:
z[1] = "B"
z # now z has only "B" entries

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "B"
 "B"
 "B"
 missing
 missing

In [27]:
levels(z) # but it remembers the levels it had (the reason is mostly performance)

2-element Vector{String}:
 "A"
 "B"

In [28]:
droplevels!(z) # this way we can clean it up
levels(z)

1-element Vector{String}:
 "B"

### Data manipulation

In [29]:
x, levels(x)

(CategoricalValue{String, UInt32}["A", "B", "B", "C"], ["A", "B", "C"])

In [30]:
x[2] = "0"
x, levels(x) # new level added at the end (works only for unordered)

(CategoricalValue{String, UInt32}["A", "0", "B", "C"], ["A", "B", "C", "0"])

In [31]:
v, levels(v)

(CategoricalValue{Int64, UInt32}[1, 2, 2, 3, 3], [1, 2, 3])

In [32]:
v[1] + v[2] # even though the underlying data is Int, we cannot operate on it

LoadError: MethodError: no method matching +(::CategoricalValue{Int64, UInt32}, ::CategoricalValue{Int64, UInt32})
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560

In [33]:
Vector{Int}(v) # you have either to retrieve the data by conversion (may be expensive)

5-element Vector{Int64}:
 1
 2
 2
 3
 3

In [34]:
unwrap(v[1]) + unwrap(v[2]) # or get a single value

3

In [35]:
unwrap.(v) # this will work for arrays witout missings

5-element Vector{Int64}:
 1
 2
 2
 3
 3

In [36]:
unwrap.(z) # also works on missing values

5-element Vector{Union{Missing, String}}:
 "B"
 "B"
 "B"
 missing
 missing

In [37]:
Vector{Union{String, Missing}}(z) # or do the conversion

5-element Vector{Union{Missing, String}}:
 "B"
 "B"
 "B"
 missing
 missing

In [38]:
recode([1,2,3,4,5,missing], 1=>10) # recode some values in an array; has also in place recode! equivalent

6-element Vector{Union{Missing, Int64}}:
 10
  2
  3
  4
  5
   missing

In [39]:
recode([1,2,3,4,5,missing], "a", 1=>10, 2=>20) # here we provided a default value for not mapped recodings

6-element Vector{Union{Missing, Int64, String}}:
 10
 20
   "a"
   "a"
   "a"
   missing

In [40]:
recode([1,2,3,4,5,missing], 1=>10, missing=>"missing") # to recode Missing you have to do it explicitly

6-element Vector{Union{Int64, String}}:
 10
  2
  3
  4
  5
   "missing"

In [41]:
t = categorical([1:5; missing])
t, levels(t)

(Union{Missing, CategoricalValue{Int64, UInt32}}[1, 2, 3, 4, 5, missing], [1, 2, 3, 4, 5])

In [42]:
recode!(t, [1,3]=>2)
t, levels(t) # note that the levels are dropped after recode

(Union{Missing, CategoricalValue{Int64, UInt32}}[2, 2, 2, 4, 5, missing], [2, 4, 5])

In [43]:
t = categorical([1,2,3], ordered=true)
levels(recode(t, 2=>0, 1=>-1)) # and if you introduce a new levels they are added at the end in the order of appearance

3-element Vector{Int64}:
  3
  0
 -1

In [44]:
t = categorical([1,2,3,4,5], ordered=true) # when using default it becomes the last level
levels(recode(t, 300, [1,2]=>100, 3=>200))

3-element Vector{Int64}:
 100
 200
 300

### Comparisons

In [45]:
x = categorical([1,2,3])
xs = [x, categorical(x), categorical(x, ordered=true), categorical(x, ordered=true)]
levels!(xs[2], [3,2,1])
levels!(xs[4], [2,3,1])
[a == b for a in xs, b in xs] # all are equal - comparison only by contents

4×4 Matrix{Bool}:
 1  1  1  1
 1  1  1  1
 1  1  1  1
 1  1  1  1

In [46]:
signature(x::CategoricalArray) = (x, levels(x), isordered(x)) # this is actually the full signature of CategoricalArray
# all are different, notice that x[1] and x[2] are unordered but have a different order of levels
[signature(a) == signature(b) for a in xs, b in xs]

4×4 Matrix{Bool}:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1

In [47]:
x[1] < x[2] # you cannot compare elements of unordered CategoricalArray

LoadError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [48]:
t[1] < t[2] # but you can do it for an ordered one

true

In [49]:
isless(x[1], x[2]) # isless works within the same CategoricalArray even if it is not ordered

true

In [50]:
y = deepcopy(x) # but not across categorical arrays
isless(x[1], y[2])

true

In [51]:
isless(unwrap(x[1]), unwrap(y[2])) # you can use get to make a comparison of the contents of CategoricalArray

true

In [52]:
x[1] == y[2] # equality tests works OK across CategoricalArrays

false

### Categorical columns in a DataFrame

In [53]:
df = DataFrame(x = 1:3, y = 'a':'c', z = ["a","b","c"])

,x,y,z
,Int64,Char,String
1,1,a,a
2,2,b,b
3,3,c,c


Convert all string columns to categorical in-place

In [54]:
transform!(df, names(df, String) => categorical, renamecols=false)

,x,y,z
,Int64,Char,Cat…
1,1,a,a
2,2,b,b
3,3,c,c


In [55]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,x,2.0,1,2.0,3,0,Int64
2,y,,a,,c,0,Char
3,z,,a,,c,0,"CategoricalValue{String, UInt32}"
